# Shaastra 2022

# Machine Learning - A project based approach 

## Prediction of attrition - if a particular employee leave the company or not 

### 14 January 2022


#### Session 2: Classification projects
#### Trainers:
Abhijit Rathod and Sai Shashank 

### Install required modules here 

####  To install catboost use pip install catboost

#### To install xgboost: If you are using anaconda use conda install -c anaconda py-xgboost OR pip install xgboost

#### To install pandas Profiling 
##### 1. pip install pandas-profiling
##### 2. pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
##### 3. conda install -c conda-forge pandas-profiling


In [1]:
# Block reserved for any installations 




## Import Python Modules, and Required Libraries 
##### Some modules and libraries are not used in this project but are given here for reader's reference (for use in other projects)

In [5]:
#Essential modules 
import pandas as pd
import numpy as np

#Visualisation libraries 
import seaborn as sns
import matplotlib.pyplot as plt

#Encoding 
from sklearn.preprocessing import LabelEncoder
#import category_encoders as ce 

#Normalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from scipy import stats

#Model Building

from sklearn import model_selection
from sklearn.model_selection import train_test_split,StratifiedKFold

##Classifiers 

# Logistic Regression
from sklearn.linear_model import LogisticRegression

#Decision tree
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

#Random Forest
from sklearn.ensemble import RandomForestClassifier

#Boosting 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
#from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

#Bagging 
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification #for bootstrapping

#KFold 
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

#Evaluation libraries 
from sklearn.metrics import recall_score
from sklearn.metrics import plot_confusion_matrix, confusion_matrix
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve

# surpress warnings
import warnings
warnings.filterwarnings('ignore')

# Timer
import timeit

## Helper Functions 
#### These helper functions are not needed as such. They are given just to show off :P. Other build in functions are also available as an alternative. 

In [6]:
def printColumnTypes(df):
    non_num_df = df.select_dtypes(include=['object'])
    num_df = df.select_dtypes(exclude=['object'])
    '''separates non-numeric and numeric columns'''
    print("Non-Numeric columns:")
    for col in non_num_df:
        print(f"{col}")
    print("")
    print("Numeric columns:")
    for col in num_df:
        print(f"{col}")

def missing_cols(df):
    '''prints out columns with its amount of missing values with its %'''
    total = 0
    for col in df.columns:
        missing_vals = df[col].isnull().sum()
        pct = df[col].isna().mean() * 100
        total += missing_vals
        if missing_vals != 0:
          print('{} => {} [{}%]'.format(col, df[col].isnull().sum(), round(pct, 2)))
    
    if total == 0:
        print("no missing values")

# PHASE 1: EXPLORING TRAINING DATA

## Upload Data

In [7]:
# df = pd.read_csv(r"F:\Shaastra Workshop\Data Sprint 56 HR Analytics\Attrition_training_data.csv")
df = pd.read_csv(r"https://raw.githubusercontent.com/gudiseva/IIT_Madras_Shaastra/main/Attrition_training_data.csv")
df

# df = df.sample(frac = 0.1)    ## Remove comment if you want to use this line 
                                ## If dataset is huge (more than 1,00,000 rows, take sample data for validation purpose. 
                                ## frac = 0.1 means 10 % data is used)

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,25,Travel_Rarely,685,Research & Development,1,3,Life Sciences,1,350,1,Female,62,3,2,Manufacturing Director,3,Married,4898,7505,0,Y,No,12,3,4,80,2,5,3,3,4,2,1,2,0
1,44,Travel_Rarely,1376,Human Resources,1,2,Medical,1,1098,2,Male,91,2,3,Human Resources,1,Married,10482,2326,9,Y,No,14,3,4,80,1,24,1,3,20,6,3,6,1
2,27,Travel_Rarely,135,Research & Development,17,4,Life Sciences,1,1405,4,Female,51,3,1,Research Scientist,3,Single,2394,25681,1,Y,Yes,13,3,4,80,0,8,2,3,8,2,7,7,1
3,40,Travel_Frequently,580,Sales,5,4,Life Sciences,1,729,4,Male,48,2,3,Sales Executive,1,Married,10475,23772,5,Y,Yes,21,4,3,80,1,20,2,3,18,13,1,12,0
4,24,Travel_Rarely,477,Research & Development,24,3,Medical,1,1173,4,Male,49,3,1,Laboratory Technician,2,Single,3597,6409,8,Y,No,22,4,4,80,0,6,2,3,4,3,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,34,Non-Travel,1381,Sales,4,4,Marketing,1,523,3,Female,72,3,2,Sales Executive,3,Married,6538,12740,9,Y,No,15,3,1,80,1,6,3,3,3,2,1,2,0
1172,19,Travel_Rarely,1181,Research & Development,3,1,Medical,1,201,2,Female,79,3,1,Laboratory Technician,2,Single,1483,16102,1,Y,No,14,3,4,80,0,1,3,3,1,0,0,0,0
1173,19,Travel_Rarely,645,Research & Development,9,2,Life Sciences,1,1193,3,Male,54,3,1,Research Scientist,1,Single,2552,7172,1,Y,No,25,4,3,80,0,1,4,3,1,1,0,0,0
1174,31,Travel_Frequently,1327,Research & Development,3,4,Medical,1,337,2,Male,73,3,3,Research Director,3,Divorced,13675,13523,9,Y,No,12,3,1,80,1,9,3,3,2,2,2,2,0


## Data Exploration Before Data Cleaning and Preprocessing 

##### Caution: It is not recommended to run this block if your laptop has low memory

In [9]:
## Magic code

## This gives entire profiling of data with visual representation in the form of an html page.

## Note: It is not recommended to run this block, if the laptop is occupied with some other tasks 
## as it requires a huge amount of memory. 

from pandas_profiling import ProfileReport
prof = ProfileReport(df)
prof

TypeError: ignored

In [10]:
prof

NameError: ignored

## Dimension of the dataset

In [11]:
print('Dimension of dataframe is', df.shape)

Dimension of dataframe is (1176, 35)


## Data types of each column

In [12]:
## method 1

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1176 entries, 0 to 1175
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1176 non-null   int64 
 1   BusinessTravel            1176 non-null   object
 2   DailyRate                 1176 non-null   int64 
 3   Department                1176 non-null   object
 4   DistanceFromHome          1176 non-null   int64 
 5   Education                 1176 non-null   int64 
 6   EducationField            1176 non-null   object
 7   EmployeeCount             1176 non-null   int64 
 8   EmployeeNumber            1176 non-null   int64 
 9   EnvironmentSatisfaction   1176 non-null   int64 
 10  Gender                    1176 non-null   object
 11  HourlyRate                1176 non-null   int64 
 12  JobInvolvement            1176 non-null   int64 
 13  JobLevel                  1176 non-null   int64 
 14  JobRole                 

In [13]:
## Method 2

printColumnTypes(df)

Non-Numeric columns:
BusinessTravel
Department
EducationField
Gender
JobRole
MaritalStatus
Over18
OverTime

Numeric columns:
Age
DailyRate
DistanceFromHome
Education
EmployeeCount
EmployeeNumber
EnvironmentSatisfaction
HourlyRate
JobInvolvement
JobLevel
JobSatisfaction
MonthlyIncome
MonthlyRate
NumCompaniesWorked
PercentSalaryHike
PerformanceRating
RelationshipSatisfaction
StandardHours
StockOptionLevel
TotalWorkingYears
TrainingTimesLastYear
WorkLifeBalance
YearsAtCompany
YearsInCurrentRole
YearsSinceLastPromotion
YearsWithCurrManager
Attrition


##### Observation: From the information table above, we can see that
##### 1. There are 28 features in the dataset 
##### 2. Non-Numeric columns:
BusinessTravel, Department, EducationField, Gender, JobRole, MaritalStatus, Over18, OverTime

## Checking number of unique values in each column 

In [14]:
df.nunique()

Age                           43
BusinessTravel                 3
DailyRate                    785
Department                     3
DistanceFromHome              29
Education                      5
EducationField                 6
EmployeeCount                  1
EmployeeNumber              1176
EnvironmentSatisfaction        4
Gender                         2
HourlyRate                    71
JobInvolvement                 4
JobLevel                       5
JobRole                        9
JobSatisfaction                4
MaritalStatus                  3
MonthlyIncome               1090
MonthlyRate                 1145
NumCompaniesWorked            10
Over18                         1
OverTime                       2
PercentSalaryHike             15
PerformanceRating              2
RelationshipSatisfaction       4
StandardHours                  1
StockOptionLevel               4
TotalWorkingYears             39
TrainingTimesLastYear          7
WorkLifeBalance                4
YearsAtCom

## Data Cleaning and Preprocessing 

### Check for missing data

In [15]:
## Method 1

df.isnull().sum()

Age                         0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSinceLastPromotion     0
YearsWithC

In [16]:
## Method 2 

missing_cols(df)      # Gives percentage of missing values

no missing values


##### Observation: No missing values

## Dealing with Missing Values 

### Alternative 1: If percentage of missing values is more than 30%, it is better to drop that column (Not a thumb rule)

### Alternative 2: Replace missing values with 'zero'

### Alternative 3: Replace missing values with 'mean' of the column 

### Alternative 4: Replace some missing values with 'mean', some with zeroes

### Alternative 5: Use advanced imputers like KNN imputer (Not in the scope of the workshop)

##### Note: axis=1 (or axis='columns') is vertical axis. To take it further, if you use pandas method drop, to remove columns or rows, if you specify axis=1 you will be removing columns. If you specify axis=0 you will be removing rows from dataset.

In [ ]:
## Alternative 1:

df = df.drop(['DataFrame Column 1','DataFrame Column 2'], axis = 1)
df

In [ ]:
## Alternative 2:
#df = df.fillna(0)

df = df.fillna(0)

In [ ]:
## Alternative 4: 

#df['DataFrame Column'] = df['DataFrame Column'].fillna(0)
#df['DataFrame Column'] = df['DataFrame Column'].fillna(mean())

df[''].fillna(int(df['NumCompaniesWorked'].mean()), inplace=True)     # When inplace = True , the data is modified in place, 
                                                                                        # which means it will return nothing 
                                                                                        # and the dataframe is now updated.
df

df['NumCompaniesWorked'] = df['NumCompaniesWorked'].fillna(0)
df

In [ ]:
## Alternative 3:
#df = df.fillna(mean())

df = df.fillna(df.mean())
df

## Check if there are missing values 

In [17]:
## Rechecking
missing_cols(df)

no missing values


## Encoding 

### Get the list of categorical Columns

In [18]:
objList = df.select_dtypes(include = "object").columns
print (objList)

Index(['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole',
       'MaritalStatus', 'Over18', 'OverTime'],
      dtype='object')


In [19]:
## Method 1: Label Encoder
#Label Encoding for object to numeric conversion

le = LabelEncoder()

for feat in objList:
    df[feat] = le.fit_transform(df[feat].astype(str))

print (df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1176 entries, 0 to 1175
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   Age                       1176 non-null   int64
 1   BusinessTravel            1176 non-null   int64
 2   DailyRate                 1176 non-null   int64
 3   Department                1176 non-null   int64
 4   DistanceFromHome          1176 non-null   int64
 5   Education                 1176 non-null   int64
 6   EducationField            1176 non-null   int64
 7   EmployeeCount             1176 non-null   int64
 8   EmployeeNumber            1176 non-null   int64
 9   EnvironmentSatisfaction   1176 non-null   int64
 10  Gender                    1176 non-null   int64
 11  HourlyRate                1176 non-null   int64
 12  JobInvolvement            1176 non-null   int64
 13  JobLevel                  1176 non-null   int64
 14  JobRole                   1176 non-null 

In [20]:
## Method 2: Find and replace 

replacement = {"Column 1":     {"element 1": 1, "element 2": 2},
                "Column 2": {"element 1": 1, "element 2": 2, "element": 3, "element 4": 4 }}

df = df.replace(replacement)
df.head()      ## head() shows first 5 rows 

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,25,2,685,1,1,3,1,1,350,1,0,62,3,2,4,3,1,4898,7505,0,0,0,12,3,4,80,2,5,3,3,4,2,1,2,0
1,44,2,1376,0,1,2,3,1,1098,2,1,91,2,3,1,1,1,10482,2326,9,0,0,14,3,4,80,1,24,1,3,20,6,3,6,1
2,27,2,135,1,17,4,1,1,1405,4,0,51,3,1,6,3,2,2394,25681,1,0,1,13,3,4,80,0,8,2,3,8,2,7,7,1
3,40,1,580,2,5,4,1,1,729,4,1,48,2,3,7,1,1,10475,23772,5,0,1,21,4,3,80,1,20,2,3,18,13,1,12,0
4,24,2,477,1,24,3,3,1,1173,4,1,49,3,1,2,2,2,3597,6409,8,0,0,22,4,4,80,0,6,2,3,4,3,1,2,0


##### To know more about categorical encoding visit https://pbpython.com/categorical-encoding.html

## Check if all the columns are numeric 

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1176 entries, 0 to 1175
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   Age                       1176 non-null   int64
 1   BusinessTravel            1176 non-null   int64
 2   DailyRate                 1176 non-null   int64
 3   Department                1176 non-null   int64
 4   DistanceFromHome          1176 non-null   int64
 5   Education                 1176 non-null   int64
 6   EducationField            1176 non-null   int64
 7   EmployeeCount             1176 non-null   int64
 8   EmployeeNumber            1176 non-null   int64
 9   EnvironmentSatisfaction   1176 non-null   int64
 10  Gender                    1176 non-null   int64
 11  HourlyRate                1176 non-null   int64
 12  JobInvolvement            1176 non-null   int64
 13  JobLevel                  1176 non-null   int64
 14  JobRole                   1176 non-null 

## Outliers Detection and Removal (Optional)

In [ ]:
## Z-score method is used to detect and remove outliers 

z = np.abs(stats.zscore(df))
print(z)

In [ ]:
## It gives indices of the rows with z-score greater than 3 i.e. rows with outliers 

threshold = 3
print(np.where(z > 3))

In [ ]:
clean_df = df[(z < 3).all(axis=1)]
clean_df

In [ ]:
print('Dimension of dataframe after cleaning is', clean_df.shape)

## Exploratory Data Analysis After Cleaning and Preprocessing - Visualization 

### Data Description 

In [22]:
df.describe()  ## Gives count, mean, std, min, max, and quartile values for each variable 

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
count,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.0,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.0,1176.000000,1176.000000,1176.000000,1176.000000,1176.0,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000
mean,36.964286,1.597789,801.496599,1.261905,9.097789,2.917517,2.262755,1.0,1038.992347,2.728741,0.598639,65.398810,2.725340,2.071429,4.441327,2.739796,1.101190,6562.642007,14241.102891,2.698980,0.0,0.274660,15.190476,3.150510,2.702381,80.0,0.789966,11.309524,2.793367,2.762755,6.978741,4.187075,2.259354,4.090986,0.161565
std,9.148817,0.673356,400.618020,0.531023,8.002540,1.022720,1.323377,0.0,599.144223,1.096404,0.490382,20.289856,0.708781,1.118510,2.454687,1.109486,0.731631,4780.847303,7153.901829,2.504369,0.0,0.446533,3.626303,0.357723,1.086017,0.0,0.851093,7.864238,1.298375,0.709095,6.066053,3.571142,3.321099,3.533579,0.368208
min,18.000000,0.000000,102.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.000000,1.000000,0.000000,30.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1009.000000,2094.000000,0.000000,0.0,0.000000,11.000000,3.000000,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,1.000000,468.250000,1.000000,2.000000,2.000000,1.000000,1.0,500.750000,2.000000,0.000000,48.000000,2.000000,1.000000,2.000000,2.000000,1.000000,2870.000000,7849.500000,1.000000,0.0,0.000000,12.000000,3.000000,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000,0.000000
50%,36.000000,2.000000,801.500000,1.000000,7.000000,3.000000,2.000000,1.0,1040.500000,3.000000,1.000000,65.000000,3.000000,2.000000,5.000000,3.000000,1.000000,4968.000000,14174.000000,2.000000,0.0,0.000000,14.000000,3.000000,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000,0.000000
75%,43.000000,2.000000,1157.250000,2.000000,14.000000,4.000000,3.000000,1.0,1574.500000,4.000000,1.000000,83.000000,3.000000,3.000000,7.000000,4.000000,2.000000,8500.000000,20344.500000,4.000000,0.0,1.000000,18.000000,3.000000,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000,0.000000
max,60.000000,2.000000,1499.000000,2.000000,29.000000,5.000000,5.000000,1.0,2065.000000,4.000000,1.000000,100.000000,4.000000,5.000000,8.000000,4.000000,2.000000,19973.000000,26999.000000,9.000000,0.0,1.000000,25.000000,4.000000,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000,1.000000


## Class Distribution

In [23]:
df['Attrition'].value_counts()

0    986
1    190
Name: Attrition, dtype: int64

In [24]:
sns.countplot(x = 'Attrition', data = df, palette="Set1");

###### Observation: Data is imbalanced as proportion of employees staying with the company is extremely higher than employees planning to leave the company. 

### Histograms 

In [25]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

sns.histplot(df, x = 'BusinessTravel', discrete = True, ax = axs[0]);
sns.histplot(df, x = 'Gender', discrete = True, ax = axs[1]);

In [27]:
df['BusinessTravel'].value_counts()

2    828
1    223
0    125
Name: BusinessTravel, dtype: int64

##### Frequently: 
##### Rarely: 
##### : 

##### Observation:  

In [28]:
df['Gender'].value_counts()

1    704
0    472
Name: Gender, dtype: int64

##### Male employees: 
##### Female employees: 

##### Observation: Proportion of male employees is significantly higher than female employees. 

In [29]:
fig, axs = plt.subplots(1,2, figsize=(10, 5))

sns.histplot(x = 'Age', data = df, discrete = True, ax = axs[0]);
sns.histplot(x = 'YearsAtCompany', data = df, discrete = True, ax = axs[1]);

In [30]:
fig, axs = plt.subplots(1,2, figsize=(10, 5))

sns.histplot(x = 'Education', data = df, discrete = True, ax = axs[0]);
sns.histplot(x = 'EducationField', data = df, discrete = True, ax = axs[1]);

## Would you like to try?

In [31]:
fig, axs = plt.subplots(1,2, figsize=(10, 5))

sns.histplot(x = '_______', data = df, discrete = True, ax = axs[0]);
sns.histplot(x = '_______', data = df, discrete = True, ax = axs[1]);

ValueError: ignored

In [32]:
# Consider EnvironmentSatisfaction, JobSatisfaction, WorkLifeBalance,JobInvolvement

fig, axs = plt.subplots(2,2, figsize=(16, 10))

#sns.histplot(x = 'Education', data = df, discrete = True, ax = axs[0][0]);
#sns.histplot(x = 'EducationField', data = df, discrete = True, ax = axs[0][1]);
#sns.histplot(x = 'EducationField', data = df, discrete = True, ax = axs[1][0]);
#sns.histplot(x = 'EducationField', data = df, discrete = True, ax = axs[1][1]);

## Box Plots    
#### To identify outliers 

In [33]:
sns.set_theme(style="whitegrid")
ax = sns.boxplot(x=df["Age"])

## Feature Selection 

### Feature Selection is the process where you automatically or manually select those features which contribute most to your prediction variable or output in which you are interested in.
### Having irrelevant features in your data can decrease the accuracy of the models and make your model learn based on irrelevant features.

### To know more visit: https://towardsdatascience.com/feature-selection-techniques-in-machine-learning-with-python-f24e7da3f36e


### Method 1: Correlation Matrix 

In [34]:
plt.figure(figsize = (12,12))
corrmat = df.corr()
ax = sns.heatmap(corrmat, vmax=1, square=True);

In [35]:
numeric_feature_cols = [x for x in df.columns if x not in ['Attrition']]
#cat_feature_cols = []
target_col = ['Attrition']

In [36]:
import scipy.stats 

corrs = []
for col in numeric_feature_cols:
    corr = scipy.stats.spearmanr(df['Attrition'], df[col])
    corrs.append({
        'feature': col,
        'correlation': corr[0],
        'correlation_p_value': corr[1]
    })
    
pd.DataFrame(corrs).sort_values('correlation')

,feature,correlation,correlation_p_value
17,MonthlyIncome,-0.207388,6.840333e-13
13,JobLevel,-0.190423,4.593898e-11
30,YearsAtCompany,-0.185196,1.558087e-10
27,TotalWorkingYears,-0.185160,1.571115e-10
31,YearsInCurrentRole,-0.176783,1.033996e-09
33,YearsWithCurrManager,-0.173101,2.301248e-09
0,Age,-0.160897,2.890453e-08
26,StockOptionLevel,-0.157777,5.358786e-08
9,EnvironmentSatisfaction,-0.114840,7.911927e-05
12,JobInvolvement,-0.105137,3.042876e-04


##### Features are arranged in the ascending order of correlation with the Attrition. Monthly income has the highest correlation with 'age_group'. Higher the correlation (positive or negative) with target variable, more the importance of the feature. This method is not very reliable as there is a possibity of interaction between variables. 

## Data preparation for Modelling 

### X and Y division

#### Drop irrelevant columns here. We generally drop columns 
##### 1. With unique id; In this case, EmployeeID. 
##### 2. With more than 30% missing values 
##### 3. More than 70-80% unique values 
##### 4. Only 1 unique value. Example: StandardHours
##### 5. Very low correlation with target variable 

In [37]:
X = df.drop(['Attrition'], axis=1)     

y = df['Attrition']

In [38]:
X

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,25,2,685,1,1,3,1,1,350,1,0,62,3,2,4,3,1,4898,7505,0,0,0,12,3,4,80,2,5,3,3,4,2,1,2
1,44,2,1376,0,1,2,3,1,1098,2,1,91,2,3,1,1,1,10482,2326,9,0,0,14,3,4,80,1,24,1,3,20,6,3,6
2,27,2,135,1,17,4,1,1,1405,4,0,51,3,1,6,3,2,2394,25681,1,0,1,13,3,4,80,0,8,2,3,8,2,7,7
3,40,1,580,2,5,4,1,1,729,4,1,48,2,3,7,1,1,10475,23772,5,0,1,21,4,3,80,1,20,2,3,18,13,1,12
4,24,2,477,1,24,3,3,1,1173,4,1,49,3,1,2,2,2,3597,6409,8,0,0,22,4,4,80,0,6,2,3,4,3,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,34,0,1381,2,4,4,2,1,523,3,0,72,3,2,7,3,1,6538,12740,9,0,0,15,3,1,80,1,6,3,3,3,2,1,2
1172,19,2,1181,1,3,1,3,1,201,2,0,79,3,1,2,2,2,1483,16102,1,0,0,14,3,4,80,0,1,3,3,1,0,0,0
1173,19,2,645,1,9,2,1,1,1193,3,1,54,3,1,6,1,2,2552,7172,1,0,0,25,4,3,80,0,1,4,3,1,1,0,0
1174,31,1,1327,1,3,4,3,1,337,2,1,73,3,3,5,3,0,13675,13523,9,0,0,12,3,1,80,1,9,3,3,2,2,2,2


In [39]:
y

0       0
1       1
2       1
3       0
4       0
       ..
1171    0
1172    0
1173    0
1174    0
1175    0
Name: Attrition, Length: 1176, dtype: int64

### Data Normalization (Optional)

In [40]:
column_maxes = X.max()
X_max = column_maxes.max()
column_mins = X.min()
X_min = column_mins.min()
normalized_X = (X - X_min) / (X_max - X_min)

normalized_X

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0.000926,0.000074,0.025371,0.000037,0.000037,0.000111,0.000037,0.000037,0.012963,0.000037,0.000000,0.002296,0.000111,0.000074,0.000148,0.000111,0.000037,0.181414,0.277973,0.000000,0.0,0.000000,0.000444,0.000111,0.000148,0.002963,0.000074,0.000185,0.000111,0.000111,0.000148,0.000074,0.000037,0.000074
1,0.001630,0.000074,0.050965,0.000000,0.000037,0.000074,0.000111,0.000037,0.040668,0.000074,0.000037,0.003370,0.000074,0.000111,0.000037,0.000037,0.000037,0.388237,0.086151,0.000333,0.0,0.000000,0.000519,0.000111,0.000148,0.002963,0.000037,0.000889,0.000037,0.000111,0.000741,0.000222,0.000111,0.000222
2,0.001000,0.000074,0.005000,0.000037,0.000630,0.000148,0.000037,0.000037,0.052039,0.000148,0.000000,0.001889,0.000111,0.000037,0.000222,0.000111,0.000074,0.088670,0.951183,0.000037,0.0,0.000037,0.000481,0.000111,0.000148,0.002963,0.000000,0.000296,0.000074,0.000111,0.000296,0.000074,0.000259,0.000259
3,0.001482,0.000037,0.021482,0.000074,0.000185,0.000148,0.000037,0.000037,0.027001,0.000148,0.000037,0.001778,0.000074,0.000111,0.000259,0.000037,0.000037,0.387977,0.880477,0.000185,0.0,0.000037,0.000778,0.000148,0.000111,0.002963,0.000037,0.000741,0.000074,0.000111,0.000667,0.000481,0.000037,0.000444
4,0.000889,0.000074,0.017667,0.000037,0.000889,0.000111,0.000111,0.000037,0.043446,0.000148,0.000037,0.001815,0.000111,0.000037,0.000074,0.000074,0.000074,0.133227,0.237379,0.000296,0.0,0.000000,0.000815,0.000148,0.000148,0.002963,0.000000,0.000222,0.000074,0.000111,0.000148,0.000111,0.000037,0.000074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,0.001259,0.000000,0.051150,0.000074,0.000148,0.000148,0.000074,0.000037,0.019371,0.000111,0.000000,0.002667,0.000111,0.000074,0.000259,0.000111,0.000037,0.242157,0.471869,0.000333,0.0,0.000000,0.000556,0.000111,0.000037,0.002963,0.000037,0.000222,0.000111,0.000111,0.000111,0.000074,0.000037,0.000074
1172,0.000704,0.000074,0.043742,0.000037,0.000111,0.000037,0.000111,0.000037,0.007445,0.000074,0.000000,0.002926,0.000111,0.000037,0.000074,0.000074,0.000074,0.054928,0.596392,0.000037,0.0,0.000000,0.000519,0.000111,0.000148,0.002963,0.000000,0.000037,0.000111,0.000111,0.000037,0.000000,0.000000,0.000000
1173,0.000704,0.000074,0.023890,0.000037,0.000333,0.000074,0.000037,0.000037,0.044187,0.000111,0.000037,0.002000,0.000111,0.000037,0.000222,0.000037,0.000074,0.094522,0.265639,0.000037,0.0,0.000000,0.000926,0.000148,0.000111,0.002963,0.000000,0.000037,0.000148,0.000111,0.000037,0.000037,0.000000,0.000000
1174,0.001148,0.000037,0.049150,0.000037,0.000111,0.000148,0.000111,0.000037,0.012482,0.000074,0.000037,0.002704,0.000111,0.000111,0.000185,0.000111,0.000000,0.506500,0.500870,0.000333,0.0,0.000000,0.000444,0.000111,0.000037,0.002963,0.000037,0.000333,0.000111,0.000111,0.000074,0.000074,0.000074,0.000074


##### This data normalization is just a demonstration of how data is normalised.
##### Normalization doesn't have much impact on decision tree algorithms. Hence normalized data is not used in prediction models. 

## Data Scaling 

In [41]:
st_x= StandardScaler()    
X = st_x.fit_transform(X)    

# PHASE 2: ALGORTIHMS ON TRAINING DATA

# Train and Validation Division

#### test_size = 0.25 means entire dataset is divided into 2 sets - Train data = 75% and Test/Validation data = 25%

In [42]:
X_train, X_val, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)
X_train.shape, X_test.shape

NameError: ignored

# Different Algortihms 

### 1. Decision Tree Classifier

In [ ]:
# Build and fit the model 

clf_gini = DecisionTreeClassifier(criterion='gini', random_state=0)
clf_gini.fit(X_train, y_train)

In [ ]:
# Predict 

y_pred_gini = clf_gini.predict(X_val)

In [ ]:
# Plot confusion matrix 

cm = confusion_matrix(y_test, y_pred_gini)
plot_confusion_matrix(clf_gini, X_val, y_test)

In [ ]:
result = pd.crosstab(y_test, y_pred_gini, rownames=['Actual Result'], colnames=['Predicted Result'])
print("Confusion Matrix:\n")
print(result)
result1 = classification_report(y_test, y_pred_gini)
print("\nClassification Report:\n",)
print (result1)
result2 = accuracy_score(y_test, y_pred_gini)
print("Accuracy of the model:",result2)

In [ ]:
# Cross Validation using KFold

scores = cross_validate(clf_gini, X_val, y_test,cv=KFold(n_splits=5))
k=scores['test_score']
print("K fold scores are:", k)
print("Average of k fold scores is:", np.mean(k, axis = None))

## 2. Random Forest 

### 2a. Random Forest with default Parameters 

In [43]:
## Random Forest Trees with default parameters (i.e. 100 Decision Trees)

rfc = RandomForestClassifier(random_state=0, class_weight='balanced')
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_val)

In [44]:
cm = confusion_matrix(y_test, y_pred)
plot_confusion_matrix(rfc, X_val, y_test)

In [45]:
result = pd.crosstab(y_test, y_pred, rownames=['Actual Result'], colnames=['Predicted Result'])
print("Confusion Matrix:\n")
print(result)
result1 = classification_report(y_test, y_pred)
print("\nClassification Report:\n",)
print (result1)
result2 = accuracy_score(y_test, y_pred)
print("Accuracy of the model:",result2)

Confusion Matrix:

Predicted Result    0  1
Actual Result           
0                 250  0
1                  38  6

Classification Report:

              precision    recall  f1-score   support

           0       0.87      1.00      0.93       250
           1       1.00      0.14      0.24        44

    accuracy                           0.87       294
   macro avg       0.93      0.57      0.58       294
weighted avg       0.89      0.87      0.83       294

Accuracy of the model: 0.8707482993197279


In [46]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average='weighted')

0.8261973042004906

In [47]:
# Cross Validation using KFold 

scores = cross_validate(rfc, X_val, y_test,cv=KFold(n_splits=5))
k=scores['test_score']
print("K fold scores are:", k)
print("Average of k fold scores is:", np.mean(k, axis = None))

K fold scores are: [0.88135593 0.81355932 0.84745763 0.84745763 0.87931034]
Average of k fold scores is: 0.8538281706604325


### 2b. Random Forest with Parameter Tuning

#### Grid Search Cross Validation to find best parameters 

In [ ]:
## Might take a while

start = timeit.default_timer()

rfc_best = RandomForestClassifier()
parameters = {
    "n_estimators":[250, 500],
    "max_depth":[8, 16, None],
    "max_features": ['auto']
    }

from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(rfc_best,parameters,cv=5, scoring='f1')      # scoring = 'roc_auc', scoring = 'accuracy'
cv.fit(X_train,y_train.values.ravel())

def display(results):
    print(f'Best parameters are: {results.best_params_}')
    print("\n")
    mean_score = results.cv_results_['mean_test_score']
    std_score = results.cv_results_['std_test_score']
    params = results.cv_results_['params']
    for mean,std,params in zip(mean_score,std_score,params):
        print(f'{round(mean,3)} + or -{round(std,3)} for the {params}')

stop = timeit.default_timer()

print('Time taken to perform Gridsearch: ', stop - start)        

display(cv)

##### GridSearch Algorithm gives best parameters using the combinations of the given set of parameters. It also cross validates the accuracy and avarage accuracy for each combination is dispalyed. 

##### In this case best parameters are: 
##### n_estimators = ___
##### max_depth = ____
##### max_features = _____

In [ ]:
## Random Forest Trees with Best Parameters

rfc_best = RandomForestClassifier(n_estimators = 250, max_depth = 8, max_features = 'auto', random_state=0)

start = timeit.default_timer()

rfc_best.fit(X_train, y_train)

stop = timeit.default_timer()

print('Time taken to perform Random Forest Classifier: ', stop - start) 

y_pred_best = rfc_best.predict(X_val)

In [ ]:
## Confusion Matrix 

cm = confusion_matrix(y_test, y_pred_best)
plot_confusion_matrix(rfc_best, X_test, y_test)

In [ ]:
result = pd.crosstab(y_test, y_pred_best, rownames=['Actual Result'], colnames=['Predicted Result'])
print("Confusion Matrix:\n")
print(result)
result1 = classification_report(y_test, y_pred_best)
print("\nClassification Report:\n",)
print (result1)
result2 = accuracy_score(y_test, y_pred_best)
print("Accuracy of the model:",result2)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred_best, average='weighted')

In [ ]:
# Cross Validation using KFold 

scores = cross_validate(rfc_best, X_val, y_test,cv=KFold(n_splits=5))
k=scores['test_score']
print("K fold scores are:", k)
print("Average of k fold scores is:", np.mean(k, axis = None))

## Method 2 of Feature Selection -  Feature Scores



In [ ]:
feature_scores = pd.Series(rfc_best.feature_importances_, index=X_train.columns).sort_values(ascending=False)
feature_scores

In [ ]:
## Plot Important Features 

feature_imp = pd.DataFrame(sorted(zip(rfc_best.feature_importances_,X.columns)), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data = feature_imp.sort_values(by="Value", ascending=False)[:25], color = 'blue');

### 2c. Random Forest using SMOTE (For Imbalanced target variables) (Optional)

#### Sample code for SMOTE demonstration

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

smote = SMOTE(sampling_strategy='not majority')

In [ ]:
rfc_smote = RandomForestClassifier(random_state=0, max_depth = 16, n_estimators = 500)

In [ ]:
clf_smote = Pipeline(steps = [('sampling', smote),('classifier',rfc_smote)])

In [ ]:
clf_smote.fit(X_train, y_train)

In [ ]:
y_pred_smote = clf_smote.predict(X_val)

In [ ]:
cm = confusion_matrix(y_test, y_pred_smote)
plot_confusion_matrix(clf_smote, X_val, y_test)

In [ ]:
result = pd.crosstab(y_test, y_pred_smote, rownames=['Actual Result'], colnames=['Predicted Result'])
print("Confusion Matrix:\n")
print(result)
result1 = classification_report(y_test, y_pred_smote)
print("\nClassification Report:\n",)
print (result1)
result2 = accuracy_score(y_test, y_pred_smote)
print("Accuracy of the model:",result2)

In [ ]:
# Cross Validation using KFold 

scores = cross_validate(clf_smote, X_val, y_test,cv=KFold(n_splits=5))
k=scores['test_score']
print("K fold scores are:", k)
print("Average of k fold scores is:", np.mean(k, axis = None))

## Boosting Algorithms 

## 3. AdaBoostClassifier

In [ ]:
clf_adaboost = AdaBoostClassifier(random_state=0)
clf_adaboost.fit(X_train, y_train)

In [ ]:
y_pred_adaboost = clf_adaboost.predict(X_val)

In [ ]:
cm = confusion_matrix(y_test, y_pred_adaboost)
plot_confusion_matrix(clf_adaboost, X_val, y_test)

In [ ]:
result = pd.crosstab(y_test, y_pred_adaboost, rownames=['Actual Result'], colnames=['Predicted Result'])
print("Confusion Matrix:\n")
print(result)
result1 = classification_report(y_test, y_pred_adaboost)
print("\nClassification Report:\n",)
print (result1)
result2 = accuracy_score(y_test, y_pred_adaboost)
print("Accuracy of the model:",result2)

In [ ]:
# Cross Validation using KFold 

scores = cross_validate(clf_adaboost, X_val, y_test,cv=KFold(n_splits=5))
k=scores['test_score']
print("K fold scores are:", k)
print("Average of k fold scores is:", np.mean(k, axis = None))

## 4. Gradient Boosting 

In [ ]:
clf_grboost = GradientBoostingClassifier(random_state=0)
clf_grboost.fit(X_train, y_train)

In [ ]:
y_pred_grboost = clf_grboost.predict(X_val)

In [ ]:
cm = confusion_matrix(y_test, y_pred_grboost)
plot_confusion_matrix(clf_grboost, X_val, y_test)
print('Confusion matrix\n\n', cm)

In [ ]:
result = pd.crosstab(y_test, y_pred_grboost, rownames=['Actual Result'], colnames=['Predicted Result'])
print("Confusion Matrix:\n")
print(result)
result1 = classification_report(y_test, y_pred_grboost)
print("\nClassification Report:\n",)
print (result1)
result2 = accuracy_score(y_test, y_pred_grboost)
print("Accuracy of the model:",result2)

In [ ]:
# Cross Validation using KFold 

scores = cross_validate(clf_grboost, X_val, y_test,cv=KFold(n_splits=5))
k=scores['test_score']
print("K fold scores are:", k)
print("Average of k fold scores is:", np.mean(k, axis = None))

### Gradient Boosting with Parameter Tuning

In [ ]:
clf_grboost_best = GradientBoostingClassifier(random_state=0)
parameters = {
    "n_estimators":[100, 250, 500],
    "max_depth":[8, 16, None],
    "max_features": [0.1, 0.25, 0.5, 1.0]
      }

from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(clf_grboost_best, parameters,cv=5)
cv.fit(X_train,y_train.values.ravel())

def display(results):
    print(f'Best parameters are: {results.best_params_}')
    print("\n")
    mean_score = results.cv_results_['mean_test_score']
    std_score = results.cv_results_['std_test_score']
    params = results.cv_results_['params']
    for mean,std,params in zip(mean_score,std_score,params):
        print(f'{round(mean,3)} + or -{round(std,3)} for the {params}')
        
display(cv)

#### Gradient Boosting with best parameters 

In [ ]:
clf_grboost_best = GradientBoostingClassifier(random_state=0)
clf_grboost_best.fit(X_train, y_train)

In [ ]:
y_pred_grboost_best = clf_grboost_best.predict(X_val)

In [ ]:
cm = confusion_matrix(y_test, y_pred_grboost_best)
plot_confusion_matrix(clf_grboost_best, X_val, y_test)

In [ ]:
result = pd.crosstab(y_test, y_pred_grboost_best, rownames=['Actual Result'], colnames=['Predicted Result'])
print("Confusion Matrix:\n")
print(result)
result1 = classification_report(y_test, y_pred_grboost_best)
print("\nClassification Report:\n",)
print (result1)
result2 = accuracy_score(y_test, y_pred_grboost_best)
print("Accuracy of the model:",result2)

In [ ]:
# Cross Validation using KFold 

scores = cross_validate(clf_grboost_best, X_val, y_test,cv=KFold(n_splits=5))
k=scores['test_score']
print("K fold scores are:", k)
print("Average of k fold scores is:", np.mean(k, axis = None))

## 5. XGBoost 

In [ ]:
clf_xgb = XGBClassifier()
clf_xgb.fit(X_train, y_train)

In [ ]:
y_pred_xgb = clf_xgb.predict(X_val)

In [ ]:
cm = confusion_matrix(y_test, y_pred_xgb)
plot_confusion_matrix(clf_xgb,X_val,y_test)
print('Confusion matrix\n\n', cm)

In [ ]:
result = pd.crosstab(y_test, y_pred_xgb, rownames=['Actual Result'], colnames=['Predicted Result'])
print("Confusion Matrix:\n")
print(result)
result1 = classification_report(y_test, y_pred_xgb)
print("\nClassification Report:\n",)
print (result1)
result2 = accuracy_score(y_test, y_pred_xgb)
print("Accuracy of the model:",result2)

In [ ]:
scores = cross_validate(clf_xgb, X_val, y_test,cv=KFold(n_splits=5))
k=scores['test_score']
print("K fold scores are:", k)
print("Average of k fold scores is:", np.mean(k, axis = None))

### XGBoost with parameter tuning 

In [ ]:
clf_xgboost_best = XGBClassifier(random_state=0)
parameters = {
    "n_estimators":[100, 250, 500],
    "max_depth":[4, 5, 6],
    "max_features": [0.1, 0.25, 0.5, 1.0],
    "learning_rate": [0.1, 0.3, 0.5],
    "gamma": [0, 0.5, 1, 5]
      }

from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(clf_xgboost_best, parameters,cv=5, scoring='f1')
cv.fit(X_train,y_train.values.ravel())


def display(results):
    print(f'Best parameters are: {results.best_params_}')
    print("\n")
    mean_score = results.cv_results_['mean_test_score']
    std_score = results.cv_results_['std_test_score']
    params = results.cv_results_['params']
    for mean,std,params in zip(mean_score,std_score,params):
        print(f'{round(mean,3)} + or -{round(std,3)} for the {params}')
        
display(cv)

In [ ]:
clf_xgboost_best = XGBClassifier(random_state=0,
                                learning_rate = 0.3,
                                max_depth = 4,
                                max_features = 0.1,
                                n_estimators = 500)
clf_xgboost_best.fit(X_train, y_train)

In [ ]:
y_pred_xgboost_best = clf_xgboost_best.predict(X_val)

In [ ]:
cm = confusion_matrix(y_test, y_pred_xgboost_best)
plot_confusion_matrix(clf_xgboost_best, X_val, y_test)

In [ ]:
result = pd.crosstab(y_test, y_pred_xgboost_best, rownames=['Actual Result'], colnames=['Predicted Result'])
print("Confusion Matrix:\n")
print(result)
result1 = classification_report(y_test, y_pred_xgboost_best)
print("\nClassification Report:\n",)
print (result1)
result2 = accuracy_score(y_test, y_pred_xgboost_best)
print("Accuracy of the model:",result2)

In [ ]:
# Cross Validation using KFold 

scores = cross_validate(clf_xgboost_best, X_val, y_test,cv=KFold(n_splits=5))
k=scores['test_score']
print("K fold scores are:", k)
print("Average of k fold scores is:", np.mean(k, axis = None))

## 6. LightGBM 

In [ ]:
clf_lgbm = LGBMClassifier()
clf_lgbm.fit(X_train, y_train)

In [ ]:
y_pred_lgbm= clf_lgbm.predict(X_val)

In [ ]:
cm = confusion_matrix(y_test, y_pred_lgbm)
plot_confusion_matrix(clf_lgbm,X_val,y_test)
print('Confusion matrix\n\n', cm)

In [ ]:
result = pd.crosstab(y_test, y_pred_lgbm, rownames=['Actual Result'], colnames=['Predicted Result'])
print("Confusion Matrix:\n")
print(result)
result1 = classification_report(y_test, y_pred_lgbm)
print("\nClassification Report:\n",)
print (result1)
result2 = accuracy_score(y_test, y_pred_lgbm)
print("Accuracy of the model:",result2)

In [ ]:
scores = cross_validate(clf_lgbm, X_val, y_test,cv=KFold(n_splits=5))
k=scores['test_score']
print("K fold scores are:", k)
print("Average of k fold scores is:", np.mean(k, axis = None))

## 7. Catboost

In [ ]:
clf_catboost = CatBoostClassifier(random_state=0, class_weights = [0.15 , 0.85])      
clf_catboost.fit(X_train, y_train)



## For imbalanced data use class_weights = [ a , b ] where a aand b are class_weights - genereally between 0 to 1. 
                                                     # Give more weight to the class with fewer rows 

In [ ]:
y_pred_catboost = clf_catboost.predict(X_val)

In [ ]:
cm = confusion_matrix(y_test, y_pred_catboost)
plot_confusion_matrix(clf_catboost,X_val,y_test)
print('Confusion matrix\n\n', cm)

In [ ]:
result1 = classification_report(y_test, y_pred_catboost)
print("\nClassification Report:\n",)
print (result1)
result2 = accuracy_score(y_test, y_pred_catboost)
print("Accuracy of the model:",result2)

In [ ]:
scores = cross_validate(clf_catboost, X_val, y_test,cv=KFold(n_splits=5))
k=scores['test_score']
print("K fold scores are:", k)
print("Average of k fold scores is:", np.mean(k, axis = None))

## 8. Bagging

In [ ]:
clf_bagging = BaggingClassifier(random_state=0)
clf_bagging.fit(X_train, y_train)

In [ ]:
y_pred_bagging = clf_bagging.predict(X_val)

In [ ]:
cm = confusion_matrix(y_test, y_pred_bagging)
plot_confusion_matrix(clf_bagging,X_val,y_test)
print('Confusion matrix\n\n', cm)

In [ ]:
result = pd.crosstab(y_test, y_pred_bagging, rownames=['Actual Result'], colnames=['Predicted Result'])
print("Confusion Matrix:\n")
print(result)
result1 = classification_report(y_test, y_pred_bagging)
print("\nClassification Report:\n",)
print (result1)
result2 = accuracy_score(y_test, y_pred_bagging)
print("Accuracy of the model:",result2)

In [ ]:
# Cross Validation using KFold 

scores = cross_validate(clf_bagging, X_val, y_test,cv=KFold(n_splits=5))
k=scores['test_score']
print("K fold scores are:", k)
print("Average of k fold scores is:", np.mean(k, axis = None))

## 9. Logistic Regression

In [ ]:
start = timeit.default_timer()

lr = LogisticRegression(class_weight = 'balanced')
lr.fit(X_train, y_train)

stop = timeit.default_timer()

print('Time: ', stop - start) 

preds = lr.predict(X_val)

In [ ]:
pd.DataFrame(confusion_matrix(y_test, preds), columns=['Predicted 0', "Predicted 1"], index=['Actual 0', 'Actual 0'])

In [ ]:
result = pd.crosstab(y_test, preds, rownames=['Actual Result'], colnames=['Predicted Result'])
print("Confusion Matrix:\n")
print(result)
result1 = classification_report(y_test, preds)
print("\nClassification Report:\n",)
print (result1)
result2 = accuracy_score(y_test, preds)
print("Accuracy of the model:",result2)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()
print(f'True Positives: {tp}')
print(f'False Positives: {fp}')
print(f'True Negatives: {tn}')
print(f'False Negatives: {fn}')

In [ ]:
probas = lr.predict_proba(X_val)[:, 1]

In [ ]:
probas

In [ ]:
def get_preds(threshold, probabilities):
    return [1 if prob > threshold else 0 for prob in probabilities]

In [ ]:
roc_values = []
for thresh in np.linspace(0, 1, 100):
    preds = get_preds(thresh, probas)
    tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()
    tpr = tp/(tp+fn)
    fpr = fp/(fp+tn)
    roc_values.append([tpr, fpr])
tpr_values, fpr_values = zip(*roc_values)

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))
ax.plot(fpr_values, tpr_values)
ax.plot(np.linspace(0, 1, 100),
         np.linspace(0, 1, 100),
         label='baseline',
         linestyle='--')
plt.title('Receiver Operating Characteristic Curve', fontsize=18)
plt.ylabel('TPR', fontsize=16)
plt.xlabel('FPR', fontsize=16)
plt.legend(fontsize=12);

In [ ]:
roc_auc_score(y_test, lr.predict_proba(X_val)[:, 1])

# PHASE 3: ALGORITHMS ON TESTING DATA

In [ ]:
# df2 = pd.read_csv(r"F:\Shaastra Workshop\Data Sprint 56 HR Analytics\Attrition_testing_data.csv")
df2 = pd.read_csv(r"https://raw.githubusercontent.com/gudiseva/IIT_Madras_Shaastra/main/Attrition_testing_data.csv")
df2

## Preprocessing of Testing Data (Exactly same as Training Data)


### 1. Check column drops 
### 2. Check missing values
### 3. Check encoding 
### 4. Check scaling 
### 5. Check anything else

In [ ]:
objList = df2.select_dtypes(include = "object").columns
print (objList)

In [ ]:
## Method 1: Label Encoder
#Label Encoding for object to numeric conversion

le = LabelEncoder()

for feat in objList:
    df2[feat] = le.fit_transform(df2[feat].astype(str))

print (df2.info())

# Final Data for Testing model

In [ ]:
X_train = X
y_train = y
print (X_train.shape, y_train.shape)
print (df2.shape)                      ## X_test is replaced by df2


## Check if number of columns in X_train and df2 are equal 

# Use your best algorithm 

### 1. Copy paste your algorithm in following block
### 2. Replace X_test to df2
### 3. Change the prediction file name to test_prediction 


### Note: Final line should look like test_prediction = model_name.predict(df2)

In [ ]:
## Random Forest Trees with default parameters (i.e. 100 Decision Trees)

rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train, y_train)


test_prediction = rfc.predict(df2)

# Preparing Prediction File for downloading 

In [ ]:
from pandas import DataFrame
df3 = DataFrame(test_prediction, columns=['Attrition'])      ## Change column name as per the requirement of the submission file
df3

In [ ]:
test_prediction

In [ ]:
y_id = df2['EmployeeNumber'].to_numpy()      ## Converting into an array 
y_id

In [ ]:
Id = DataFrame(y_id, columns = ['Id'], index = None)       ## Converting into a dataframe
Id

In [ ]:
result = Id.join(df3)     ### Joining test unique id with predictions 

In [ ]:
result

# Downloading submission file 

In [ ]:
### Save the file 
### Remember to change the file name 

result.to_csv(r'F:\Shaastra Workshop\Data Sprint 56 HR Analytics\prediction_2.csv', index = False)   

## 'index = False' removes first column with auto generated index ids

## Compare your Results 

### The best model with respect to accuracy: ___________ with accuracy of ___________ %

### Write the accuracy of the model:

### Model:                                              Accuracy  

### 1. XGBoost :                                    
### 2. Gradient Boosting :                   
### 3. Light GBM :                                  
### 4. CatBoost :                                 
### 5. Random Forest :                          
### 6. Bagging :                                      
### 7. Decision Trees :                           
### 8. Adaboost :   
### 9. Logistic Regression:

## End of code